In [25]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import pandas as pd
# numpy
import numpy as np
# random
from random import shuffle
# classifier
from sklearn.linear_model import LogisticRegression

# get dataframe

In [26]:
NegTrainDataPath = "CleanData\\train-neg.txt"
PosTrainDataPath = "CleanData\\train-pos.txt"
UnLabelTrainDataPath = "CleanData\\train-unsup.txt"
testDataPath = "CleanData\\test.txt"

In [30]:
NegLabelTrainDataFrame = pd.read_csv(NegTrainDataPath,header=None)
PosLabelTrainDataFrame = pd.read_csv(PosTrainDataPath,header=None)
UnLabelTrainDataFrame = pd.read_csv(UnLabelTrainDataPath,header=None)
TestDataFrame = pd.read_csv(testDataPath,header=None)

In [31]:
print("NegTrainData = ",NegLabelTrainDataFrame.shape[0])
print("PosTrainData = ",PosLabelTrainDataFrame.shape[0])
print("UnLabelTrainData = ",UnLabelTrainDataFrame.shape[0])
print("TestData = ",TestDataFrame.shape[0])

NegTrainData =  12500
PosTrainData =  12500
UnLabelTrainData =  50000
TestData =  25000


# loadmodel

In [32]:
model = Doc2Vec.load('./Model/imdb.d2v')

#  a sample vector 

In [33]:
model['TRAIN_NEG_0']

array([ -6.62691534e-01,  -7.76247692e+00,   2.40531802e+00,
        -3.79832411e+00,   4.68630075e+00,   6.01266742e-01,
         2.65584946e+00,   2.48005986e-03,  -3.24314213e+00,
        -8.89416933e-02,   2.93580627e+00,  -7.79288113e-01,
         1.55539155e+00,  -3.08664632e+00,   6.55950308e+00,
         2.63487029e+00,   3.27819824e+00,  -7.67582417e-01,
         6.47629261e-01,  -1.07775927e+00,   3.05098981e-01,
        -2.86874801e-01,  -3.14541960e+00,   9.44287360e-01,
         1.80172253e+00,  -3.12100601e+00,  -2.05273628e+00,
        -5.92697084e-01,  -4.72062302e+00,  -6.03741312e+00,
        -3.10472775e+00,   2.27795315e+00,  -5.60661602e+00,
         3.58796453e+00,  -6.22448087e-01,   1.44651663e+00,
        -8.29010382e-02,  -1.95712864e+00,  -5.06302655e-01,
         1.62629855e+00,   2.58372545e+00,   1.21157682e+00,
        -1.70597863e+00,   1.65418839e+00,   1.13596499e+00,
         2.81472373e+00,  -5.64618492e+00,  -4.42828560e+00,
        -3.87430906e+00,

## Inspecting the Model

In [34]:
model.wv.most_similar('good')

[('decent', 0.7674194574356079),
 ('great', 0.7440007328987122),
 ('fine', 0.7338722944259644),
 ('bad', 0.7250358462333679),
 ('solid', 0.6936144232749939),
 ('nice', 0.6750378012657166),
 ('well', 0.6653556823730469),
 ('fantastic', 0.6441545486450195),
 ('terrible', 0.6301878690719604),
 ('excellent', 0.6274154186248779)]

In [35]:
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [36]:
model.wv.most_similar(positive=['woman','boy'],negative=['man'],topn=10)

[('girl', 0.8233434557914734),
 ('men', 0.6354216933250427),
 ('lady', 0.6248824596405029),
 ('child', 0.5895017385482788),
 ('kid', 0.5808984637260437),
 ('gal', 0.571243941783905),
 ('teenager', 0.560899555683136),
 ('daughter', 0.5439508557319641),
 ('sister', 0.5325612425804138),
 ('lad', 0.5323701500892639)]

In [40]:
model.docvecs.most_similar(0)

[('TRAIN_POS_6447', 0.4735139310359955),
 ('TEST_10853', 0.4671124815940857),
 ('TEST_14878', 0.4459395408630371),
 ('TRAIN_POS_9332', 0.4343237578868866),
 ('TRAIN_UNS_23216', 0.43367835879325867),
 ('TRAIN_UNS_24182', 0.43035855889320374),
 ('TRAIN_UNS_13567', 0.4298480749130249),
 ('TRAIN_UNS_9710', 0.426700234413147),
 ('TRAIN_UNS_18602', 0.42652761936187744),
 ('TEST_22134', 0.426236093044281)]

In [38]:
model["TEST_24810"]

array([ 2.95239973, -1.24969399,  3.29956985, -3.90462565,  4.61045265,
        5.51968288,  0.39832857,  4.1443119 , -1.29792786, -0.34585753,
       -4.00481272,  0.86193132,  3.09168172,  3.9264338 , -1.76124251,
       -5.4568038 ,  0.59171295,  3.06078196, -4.07117271, -0.18574995,
        3.2747879 , -0.46941936,  0.03911133, -4.64472246,  4.89604139,
       -3.14762974, -1.24758673, -0.30737233, -0.23196392,  0.84912157,
       -4.2230196 , -0.7549361 ,  4.38383293,  2.61794567, -0.55559188,
       -1.1644603 ,  1.79213178, -1.66251159, -0.5805034 ,  1.87585771,
       -3.2442956 ,  2.58748746, -6.56832647, -2.94067574,  1.32970643,
        2.68853927, -1.69721258, -1.79148626, -8.86662197,  1.20294356,
       -1.77326357, -0.10407368,  2.62547898, -0.50494879, -2.01088643,
        0.09080964,  2.30544114, -2.91932392, -4.02691031,  8.32335854,
        0.53457642,  3.04086828,  4.17223024, -0.7443428 ,  2.18706608,
       -2.46791577,  0.38441673,  8.10077477,  1.06671786, -1.57

# get vector for train

In [15]:
train_arrays = np.zeros((25000, 100))
train_labels = np.zeros(25000)
for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model[prefix_train_pos]
    train_arrays[12500 + i] = model[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

In [16]:
len(train_arrays)

25000

In [17]:
train_labels

array([ 1.,  1.,  1., ...,  0.,  0.,  0.])

## Testing Vectors

In [18]:
test_arrays = np.zeros((25000, 100))
test_labels = np.zeros(25000)
for i in range(25000):
    prefix_test = 'TEST_' + str(i)
    test_arrays[i] = model[prefix_test]

In [19]:
test_arrays

array([[-0.39829364, -1.03931415,  0.46423477, ...,  1.41197491,
         0.0731272 ,  0.40817806],
       [ 1.22825682, -0.1777443 ,  0.84348565, ...,  0.44782853,
        -0.85918063,  0.85374272],
       [-0.16876324, -0.28066337,  0.16496465, ...,  0.65222448,
         1.08759677,  0.59662902],
       ..., 
       [ 0.73282796, -0.03569606,  0.60188103, ...,  0.68542176,
        -1.02991974,  0.41950887],
       [-0.21041313, -0.68366021,  0.20423155, ...,  0.76312685,
         1.30831742,  1.06854141],
       [ 0.3202036 ,  0.23481277,  0.78861231, ..., -0.16153495,
         0.89629811,  0.06097912]])

# get data from class

In [ ]:
import Train

In [ ]:
data = GetData("./imdb2.d2v","TRAIN_POS_","TRAIN_NEG_",12500,12500,"TEST_",25000)

model = data.GetModel()

train_arrays, train_labels, test_arrays = data.GetArray()